In [199]:
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score
import pickle

In [200]:
insulin_data = pd.DataFrame(pd.read_csv('insulinData.csv'))

C:\Users\nisha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (13,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
insulin_data_p2 = pd.DataFrame(pd.read_csv('InsulinAndMealIntake670GPatient3.csv'))

C:\Users\nisha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
cgm_data = pd.DataFrame(pd.read_csv('CGMData.csv'))

C:\Users\nisha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
cgm_data_p2 = pd.DataFrame(pd.read_csv('CGMData670GPatient3.csv'))

In [7]:
match_time = []
match_date = []
for i in range(len(insulin_data)):
    if(not(math.isnan(insulin_data.loc[i, 'BWZ Carb Input (grams)']))):
        match_date.append(insulin_data.loc[i, 'Date'])
        match_time.append(insulin_data.loc[i, 'Time'])

In [8]:
match_time_p2 = []
match_date_p2 = []
for i in range(len(insulin_data_p2)):
    if(not(math.isnan(insulin_data_p2.loc[i, 'BWZ Carb Input (grams)']))):
        match_date_p2.append(insulin_data_p2.loc[i, 'Date'])
        match_time_p2.append(insulin_data_p2.loc[i, 'Time'])

In [9]:
dtl = []
for i in range(len(match_date)):
    dtl.append(str(match_date[i]) + ' ' + str(match_time[i]))

In [10]:
dtl_p2 = []
for i in range(len(match_date_p2)):
    dtl_p2.append(str(match_date_p2[i]) + ' ' + str(match_time_p2[i]))

In [11]:
insulin_data = insulin_data.iloc[::-1]

In [12]:
insulin_data_p2 = insulin_data_p2.iloc[::-1]

In [13]:
cgm_data = cgm_data[::-1]

In [14]:
cgm_data_p2 = cgm_data_p2.iloc[::-1]

In [15]:
for i in range(len(dtl)):
    dtl[i] = datetime.strptime(str(match_date[i]) + ' ' + str(match_time[i]), '%m/%d/%Y %H:%M:%S')


In [16]:
for i in range(len(dtl_p2)):
    dtl_p2[i] = datetime.strptime(str(match_date_p2[i]) + ' ' + str(match_time_p2[i]), '%d-%m-%Y %H:%M:%S')

In [17]:
dtl = dtl[::-1]

In [18]:
dtl_p2 = dtl_p2[::-1]

In [19]:
meal_data_dt = []
for i in range(len(dtl)-1):
    if((dtl[i] + timedelta(hours = 2)) < dtl[i+1]):
        meal_data_dt.append(dtl[i])
    if((dtl[i] + timedelta(hours = 2)) == dtl[i+1]):
        meal_data_dt.append(dtl[i+1] + timedelta(hours = -0.5))
    

In [20]:
meal_data_dt_p2 = []
for i in range(len(dtl_p2)-1):
    if((dtl_p2[i] + timedelta(hours = 2)) < dtl_p2[i+1]):
        meal_data_dt_p2.append(dtl_p2[i])
    if((dtl_p2[i] + timedelta(hours = 2)) == dtl_p2[i+1]):
        meal_data_dt_p2.append(dtl_p2[i+1] + timedelta(hours = -0.5))

In [21]:
no_meal_data_dt = []
for i in range(len(dtl)-1):
    if((dtl[i] + timedelta(hours = 4)) < dtl[i+1]):
        no_meal_data_dt.append(dtl[i] + timedelta(hours = 2))

In [22]:
no_meal_data_dt_p2 = []
for i in range(len(dtl_p2)-1):
    if((dtl_p2[i] + timedelta(hours = 4)) < dtl_p2[i+1]):
        no_meal_data_dt_p2.append(dtl_p2[i] + timedelta(hours = 2))

In [23]:
cgm_data_dt = []
for i in range(len(cgm_data)):
    cgm_data_dt.append(datetime.strptime(str(cgm_data.loc[i, 'Date']) + ' ' + str(cgm_data.loc[i, 'Time']), '%m/%d/%Y %H:%M:%S'))

In [24]:
cgm_data_dt_p2 = []
for i in range(len(cgm_data_p2)):
    cgm_data_dt_p2.append(datetime.strptime(str(cgm_data_p2.loc[i, 'Date']) + ' ' + str(cgm_data_p2.loc[i, 'Time']), '%d-%m-%Y %H:%M:%S'))

# meal data of cgm


In [25]:
cgm_tm_match_meal = []
for i in range(len(meal_data_dt)):
    for j in range(len(cgm_data_dt)):
        if(cgm_data_dt[j] < meal_data_dt[i] + timedelta(minutes = 5)):
            cgm_tm_match_meal.append((cgm_data_dt[j]))
            break

In [26]:
cgm_tm_match_meal_p2 = []
for i in range(len(meal_data_dt_p2)):
    for j in range(len(cgm_data_dt_p2)):
        if(cgm_data_dt_p2[j] < meal_data_dt_p2[i] + timedelta(minutes = 5)):
            cgm_tm_match_meal_p2.append((cgm_data_dt_p2[j]))
            break

# no meal data of CGM


In [27]:
cgm_tm_match_no_meal = []
for i in range(len(no_meal_data_dt)):
    for j in range(len(cgm_data_dt)):
        if(cgm_data_dt[j] < no_meal_data_dt[i] + timedelta(minutes = 5)):
            cgm_tm_match_no_meal.append((cgm_data_dt[j]))
            break

In [28]:
cgm_tm_match_no_meal_p2 = []
for i in range(len(no_meal_data_dt_p2)):
    for j in range(len(cgm_data_dt_p2)):
        if(cgm_data_dt_p2[j] < no_meal_data_dt_p2[i] + timedelta(minutes = 5)):
            cgm_tm_match_no_meal_p2.append((cgm_data_dt_p2[j]))
            break

In [29]:
cgm_data['timestamp'] = cgm_data_dt[::-1]

In [30]:
cgm_data_p2['timestamp'] = cgm_data_dt_p2[::-1]

# Meal data cgm values

In [31]:
temp_index_list = []
for i in cgm_tm_match_meal:
    temp_index_list.append(cgm_data[cgm_data['timestamp'] == i].index[0])

In [32]:
temp_index_list_p2 = []
for i in cgm_tm_match_meal_p2:
    temp_index_list_p2.append(cgm_data_p2[cgm_data_p2['timestamp'] == i].index[0])

In [33]:
cgm_value_stretch_meal = []
for i in range(1,len(temp_index_list)):
    cgm_value_one_stretch_meal = []
    for j in range(30):
        cgm_value_one_stretch_meal.append(cgm_data.loc[(temp_index_list[i] + 6 - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_meal).all())):
        cgm_value_stretch_meal.append(cgm_value_one_stretch_meal)


In [34]:
cgm_value_stretch_meal_p2 = []
for i in range(1,len(temp_index_list_p2)):
    cgm_value_one_stretch_meal_p2 = []
    for j in range(30):
        cgm_value_one_stretch_meal_p2.append(cgm_data_p2.loc[(temp_index_list_p2[i] + 6 - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_meal_p2).all())):
        cgm_value_stretch_meal_p2.append(cgm_value_one_stretch_meal_p2)

In [35]:
for i in range(len(cgm_value_stretch_meal)):
    for j in range(len(cgm_value_stretch_meal[i])):
        if(np.isnan(cgm_value_stretch_meal[i][j])):
            cgm_value_stretch_meal[i][j] = int(np.nanmean(cgm_value_stretch_meal[i]))

In [36]:
for i in range(len(cgm_value_stretch_meal_p2)):
    for j in range(len(cgm_value_stretch_meal_p2[i])):
        if(np.isnan(cgm_value_stretch_meal_p2[i][j])):
            cgm_value_stretch_meal_p2[i][j] = int(np.nanmean(cgm_value_stretch_meal_p2[i]))

In [37]:
cgm_value_stretch_meal_df = pd.DataFrame(cgm_value_stretch_meal)

In [38]:
cgm_value_stretch_meal_df_p2 = pd.DataFrame(cgm_value_stretch_meal_p2)

# no meal data cgm values

In [39]:
temp_index_no_meal_list = []
for i in cgm_tm_match_no_meal:
    temp_index_no_meal_list.append(cgm_data[cgm_data['timestamp'] == i].index[0])

In [40]:
temp_index_no_meal_list_p2 = []
for i in cgm_tm_match_no_meal_p2:
    temp_index_no_meal_list_p2.append(cgm_data_p2[cgm_data_p2['timestamp'] == i].index[0])

In [41]:
cgm_value_stretch_no_meal = []
for i in range(len(temp_index_no_meal_list)):
    cgm_value_one_stretch_no_meal = []
    for j in range(24):
        cgm_value_one_stretch_no_meal.append(cgm_data.loc[(temp_index_no_meal_list[i] - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_no_meal).all())):
        cgm_value_stretch_no_meal.append(cgm_value_one_stretch_no_meal)

In [42]:
cgm_value_stretch_no_meal_p2 = []
for i in range(len(temp_index_no_meal_list_p2)):
    cgm_value_one_stretch_no_meal_p2 = []
    for j in range(24):
        cgm_value_one_stretch_no_meal_p2.append(cgm_data_p2.loc[(temp_index_no_meal_list_p2[i] - j),'Sensor Glucose (mg/dL)'])
    if(not(np.isnan(cgm_value_one_stretch_no_meal_p2).all())):
        cgm_value_stretch_no_meal_p2.append(cgm_value_one_stretch_no_meal_p2)

In [43]:
for i in range(len(cgm_value_stretch_no_meal)):
    for j in range(len(cgm_value_stretch_no_meal[i])):
        if(np.isnan(cgm_value_stretch_no_meal[i][j])):
            cgm_value_stretch_no_meal[i][j] = (np.nanmean(cgm_value_stretch_no_meal[i]))

In [44]:
for i in range(len(cgm_value_stretch_no_meal_p2)):
    for j in range(len(cgm_value_stretch_no_meal_p2[i])):
        if(np.isnan(cgm_value_stretch_no_meal_p2[i][j])):
            cgm_value_stretch_no_meal_p2[i][j] = (np.nanmean(cgm_value_stretch_no_meal_p2[i]))

In [45]:
cgm_value_stretch_no_meal_df = pd.DataFrame(cgm_value_stretch_no_meal)

In [46]:
cgm_value_stretch_no_meal_df_p2 = pd.DataFrame(cgm_value_stretch_no_meal_p2)

# Feature Engineering

# CGM Velocity, tmax

In [47]:
cgm_velocity_meal_p1 = np.diff(cgm_value_stretch_meal_df)

In [48]:
cgm_velocity_meal_p2 = np.diff(cgm_value_stretch_meal_df_p2)

In [49]:
cgm_velocity_no_meal_p1 = np.diff(cgm_value_stretch_no_meal_df)

In [50]:
cgm_velocity_no_meal_p2 = np.diff(cgm_value_stretch_no_meal_df_p2)

In [51]:
max_cgm_velocity_meal_p1 = []
for i in range(len(cgm_velocity_meal_p1)):
    max_cgm_velocity_meal_p1.append([max(cgm_velocity_meal_p1[i]), np.argmax(cgm_velocity_meal_p1[i])*5])

In [52]:
max_cgm_velocity_meal_p2 = []
for i in range(len(cgm_velocity_meal_p2)):
    max_cgm_velocity_meal_p2.append([max(cgm_velocity_meal_p2[i]), np.argmax(cgm_velocity_meal_p2[i])*5])

In [53]:
max_cgm_velocity_no_meal_p1 = []
for i in range(len(cgm_velocity_no_meal_p1)):
    max_cgm_velocity_no_meal_p1.append([max(cgm_velocity_no_meal_p1[i]), np.argmax(cgm_velocity_no_meal_p1[i])*5])

In [54]:
max_cgm_velocity_no_meal_p2 = []
for i in range(len(cgm_velocity_no_meal_p2)):
    max_cgm_velocity_no_meal_p2.append([max(cgm_velocity_no_meal_p2[i]), np.argmax(cgm_velocity_no_meal_p2[i])*5])

# CGMmax - CGMmin

In [55]:
cgm_range_meal_p1 = cgm_value_stretch_meal_df.max(axis = 1) - cgm_value_stretch_meal_df.min(axis = 1)

In [56]:
cgm_range_no_meal_p1 = cgm_value_stretch_no_meal_df.max(axis = 1) - cgm_value_stretch_no_meal_df.min(axis = 1)

In [57]:
cgm_range_meal_p2 = cgm_value_stretch_meal_df_p2.max(axis = 1) - cgm_value_stretch_meal_df_p2.min(axis = 1)

In [58]:
cgm_range_no_meal_p2 = cgm_value_stretch_no_meal_df_p2.max(axis = 1) - cgm_value_stretch_no_meal_df_p2.min(axis = 1)

# Important duration

In [59]:
imp_duration_meal_p1 = (cgm_value_stretch_meal_df.idxmax(axis = 1))*5

In [60]:
imp_duration_no_meal_p1 = (cgm_value_stretch_no_meal_df.idxmax(axis = 1))*5

In [61]:
imp_duration_meal_p2 = (cgm_value_stretch_meal_df_p2.idxmax(axis = 1))*5

In [62]:
imp_duration_no_meal_p2 = (cgm_value_stretch_no_meal_df_p2.idxmax(axis = 1))*5

# FFT

In [63]:
fft_meal_p1 = []
temp1 = []
#for i in range(len(cgm_value_stretch_meal_df)):
temp1 = abs(np.fft.fft(cgm_value_stretch_meal_df)) #[i:i+1])).flatten()
for i in range(len(temp1)):
    temp1[i].sort()
    fft_meal_p1.append([temp1[i][-2], temp1[i][-4]])

In [64]:
fft_no_meal_p1 = []
temp1 = []
#for i in range(len(cgm_value_stretch_meal_df)):
temp1 = abs(np.fft.fft(cgm_value_stretch_no_meal_df)) #[i:i+1])).flatten()
for i in range(len(temp1)):
    temp1[i].sort()
    fft_no_meal_p1.append([temp1[i][-2], temp1[i][-4]])

In [65]:
fft_meal_p2 = []
temp1 = []
#for i in range(len(cgm_value_stretch_meal_df)):
temp1 = abs(np.fft.fft(cgm_value_stretch_meal_df_p2)) #[i:i+1])).flatten()
for i in range(len(temp1)):
    temp1[i].sort()
    fft_meal_p2.append([temp1[i][-2], temp1[i][-4]])

In [66]:
fft_no_meal_p2 = []
temp1 = []
#for i in range(len(cgm_value_stretch_meal_df)):
temp1 = abs(np.fft.fft(cgm_value_stretch_no_meal_df_p2)) #[i:i+1])).flatten()
for i in range(len(temp1)):
    temp1[i].sort()
    fft_no_meal_p2.append([temp1[i][-2], temp1[i][-4]])

# windowed mean

In [67]:
win_mean_meal_p1 = []
win_mean_meal_p1.append([cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[8], cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[11], cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[14], cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[17], cgm_value_stretch_meal_df.rolling(3, win_type='triang', axis = 1).mean()[20]])

In [68]:
win_mean_no_meal_p1 = []
win_mean_no_meal_p1.append([cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[8], cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[11], cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[14], cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[17], cgm_value_stretch_no_meal_df.rolling(3, win_type='triang', axis = 1).mean()[20]])

In [69]:
win_mean_meal_p2 = []
win_mean_meal_p2.append([cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[8], cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[11], cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[14], cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[17], cgm_value_stretch_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[20]])

In [70]:
win_mean_no_meal_p2 = []
win_mean_no_meal_p2.append([cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[8], cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[11], cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[14], cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[17], cgm_value_stretch_no_meal_df_p2.rolling(3, win_type='triang', axis = 1).mean()[20]])

In [71]:
feat_win_meal_p2 = []
for j in range(5):
    temp = []
    for i in range(len(win_mean_meal_p2[0][j])):
        temp.append(win_mean_meal_p2[0][j][i])
    feat_win_meal_p2.append(temp)
feat_win_meal_p2 = np.array(feat_win_meal_p2).T

In [72]:
feat_win_meal_p1 = []
for j in range(5):
    temp = []
    for i in range(len(win_mean_meal_p1[0][j])):
        temp.append(win_mean_meal_p1[0][j][i])
    feat_win_meal_p1.append(temp)
feat_win_meal_p1 = np.array(feat_win_meal_p1).T

In [73]:
feat_win_no_meal_p1 = []
for j in range(5):
    temp = []
    for i in range(len(win_mean_no_meal_p1[0][j])):
        temp.append(win_mean_no_meal_p1[0][j][i])
    feat_win_no_meal_p1.append(temp)
feat_win_no_meal_p1 = np.array(feat_win_no_meal_p1).T

In [74]:
feat_win_no_meal_p2 = []
for j in range(5):
    temp = []
    for i in range(len(win_mean_no_meal_p2[0][j])):
        temp.append(win_mean_no_meal_p2[0][j][i])
    feat_win_no_meal_p2.append(temp)
feat_win_no_meal_p2 = np.array(feat_win_no_meal_p2).T

# Creating feature matrix

In [75]:
'''max_cgm_velocity_meal_p1 = []
for i in range(len(cgm_velocity_meal_p1)):
    max_cgm_velocity_meal_p1.append([max(cgm_velocity_meal_p1[i]), np.argmax(cgm_velocity_meal_p1[i])*5])'''
final_feature_matrix = []
feature_matrix = []
feature_matrix.append(max_cgm_velocity_meal_p1)
final_feature_matrix.append(feature_matrix[0])
final_feature_matrix = final_feature_matrix[0]

In [76]:
'''max_cgm_velocity_no_meal_p1 = []
for i in range(len(cgm_velocity_no_meal_p1)):
    max_cgm_velocity_no_meal_p1.append([max(cgm_velocity_no_meal_p1[i]), np.argmax(cgm_velocity_no_meal_p1[i])*5])'''
final_feature_matrix_no_meal_p1 = []
feature_matrix_no_meal_p1 = []
feature_matrix_no_meal_p1.append(max_cgm_velocity_no_meal_p1)
final_feature_matrix_no_meal_p1.append(feature_matrix_no_meal_p1[0])
final_feature_matrix_no_meal_p1 = final_feature_matrix_no_meal_p1[0]

In [77]:
'''max_cgm_velocity_meal_p2 = []
for i in range(len(cgm_velocity_meal_p2)):
    max_cgm_velocity_meal_p2.append([max(cgm_velocity_meal_p2[i]), np.argmax(cgm_velocity_meal_p2[i])*5])'''
final_feature_matrix_meal_p2 = []
feature_matrix_meal_p2 = []
feature_matrix_meal_p2.append(max_cgm_velocity_meal_p2)
final_feature_matrix_meal_p2.append(feature_matrix_meal_p2[0])
final_feature_matrix_meal_p2 = final_feature_matrix_meal_p2[0]

In [78]:
'''max_cgm_velocity_no_meal_p2 = []
for i in range(len(cgm_velocity_no_meal_p2)):
    max_cgm_velocity_no_meal_p2.append([max(cgm_velocity_no_meal_p2[i]), np.argmax(cgm_velocity_no_meal_p2[i])*5])'''
final_feature_matrix_no_meal_p2 = []
feature_matrix_no_meal_p2 = []
feature_matrix_no_meal_p2.append(max_cgm_velocity_no_meal_p2)
final_feature_matrix_no_meal_p2.append(feature_matrix_no_meal_p2[0])
final_feature_matrix_no_meal_p2 = final_feature_matrix_no_meal_p2[0]

In [79]:
for i in range(len(final_feature_matrix)):
    final_feature_matrix[i].append(cgm_range_meal_p1[i])

In [80]:
for i in range(len(final_feature_matrix_no_meal_p1)):
    final_feature_matrix_no_meal_p1[i].append(cgm_range_no_meal_p1[i])

In [81]:
for i in range(len(final_feature_matrix_meal_p2)):
    final_feature_matrix_meal_p2[i].append(cgm_range_meal_p2[i])

In [82]:
for i in range(len(final_feature_matrix_no_meal_p2)):
    final_feature_matrix_no_meal_p2[i].append(cgm_range_no_meal_p2[i])

# important duration for test

In [83]:
for i in range(len(final_feature_matrix_no_meal_p1)):
    final_feature_matrix_no_meal_p1[i].append(imp_duration_no_meal_p1[i])

In [84]:
for i in range(len(final_feature_matrix)):
    final_feature_matrix[i].append(imp_duration_meal_p1[i])

In [85]:
for i in range(len(final_feature_matrix_meal_p2)):
    final_feature_matrix_meal_p2[i].append(imp_duration_meal_p2[i])

In [86]:
for i in range(len(final_feature_matrix_no_meal_p2)):
    final_feature_matrix_no_meal_p2[i].append(imp_duration_no_meal_p2[i])

In [87]:
for i in range(len(final_feature_matrix)):
    final_feature_matrix[i].append(fft_meal_p1[i][0])
    final_feature_matrix[i].append(fft_meal_p1[i][1])

In [88]:
for i in range(len(final_feature_matrix_no_meal_p1)):
    final_feature_matrix_no_meal_p1[i].append(fft_no_meal_p1[i][0])
    final_feature_matrix_no_meal_p1[i].append(fft_no_meal_p1[i][1])

In [89]:
for i in range(len(final_feature_matrix_meal_p2)):
    final_feature_matrix_meal_p2[i].append(fft_meal_p2[i][0])
    final_feature_matrix_meal_p2[i].append(fft_meal_p2[i][1])

In [90]:
for i in range(len(final_feature_matrix_no_meal_p2)):
    final_feature_matrix_no_meal_p2[i].append(fft_no_meal_p2[i][0])
    final_feature_matrix_no_meal_p2[i].append(fft_no_meal_p2[i][1])

In [91]:
for i in range(len(final_feature_matrix)):
    for j in range(5):
        final_feature_matrix[i].append(feat_win_meal_p1[i][j])

In [92]:
for i in range(len(final_feature_matrix_no_meal_p1)):
    for j in range(5):
        final_feature_matrix_no_meal_p1[i].append(feat_win_no_meal_p1[i][j])

In [93]:
for i in range(len(final_feature_matrix_no_meal_p2)):
    for j in range(5):
        final_feature_matrix_no_meal_p2[i].append(feat_win_no_meal_p2[i][j])

In [94]:
for i in range(len(final_feature_matrix_meal_p2)):
    for j in range(5):
        final_feature_matrix_meal_p2[i].append(feat_win_meal_p2[i][j])

In [95]:
final_feature_matrix = (np.array(final_feature_matrix).T).T

In [96]:
final_feature_matrix_no_meal_p1 = (np.array(final_feature_matrix_no_meal_p1).T).T

In [97]:
final_feature_matrix_meal_p2 = (np.array(final_feature_matrix_meal_p2).T).T

In [98]:
final_feature_matrix_no_meal_p2 = (np.array(final_feature_matrix_no_meal_p2).T).T

In [99]:
final_feature_df = pd.DataFrame(final_feature_matrix, columns=['Max CGM velocity', 'time at max vel', 'CGM max-min', 'Important duration', 'FFT1', 'FFT2', 'Win_mean1', 'Win_mean2', 'Win_mean3', 'Win_mean4', 'Win_mean5'])

In [100]:
final_feature_df_no_meal_p1 = pd.DataFrame(final_feature_matrix_no_meal_p1, columns=['Max CGM velocity', 'time at max vel', 'CGM max-min', 'Important duration', 'FFT1', 'FFT2', 'Win_mean1', 'Win_mean2', 'Win_mean3', 'Win_mean4', 'Win_mean5'])

In [101]:
final_feature_df_meal_p2 = pd.DataFrame(final_feature_matrix_meal_p2, columns=['Max CGM velocity', 'time at max vel', 'CGM max-min', 'Important duration', 'FFT1', 'FFT2', 'Win_mean1', 'Win_mean2', 'Win_mean3', 'Win_mean4', 'Win_mean5'])

In [102]:
final_feature_df_no_meal_p2 = pd.DataFrame(final_feature_matrix_no_meal_p2, columns=['Max CGM velocity', 'time at max vel', 'CGM max-min', 'Important duration', 'FFT1', 'FFT2', 'Win_mean1', 'Win_mean2', 'Win_mean3', 'Win_mean4', 'Win_mean5'])

In [103]:
final_feature_df_no_meal_p1['meal'] = 0

In [104]:
final_feature_df['meal'] = 1

In [105]:
final_feature_df_no_meal_p2['meal'] = 0

In [106]:
final_feature_df_meal_p2['meal'] = 1

# training data 

In [107]:
final_data_p1 = pd.concat([final_feature_df[0:200], final_feature_df_no_meal_p1[0:200]])

# testing data

In [108]:
test_final_data_p2 = pd.concat([final_feature_df_meal_p2, final_feature_df_no_meal_p2])

In [156]:

X_train,X_test,y_train,y_test = train_test_split(final_data_p1.iloc[:,:-1], final_data_p1['meal'], test_size=0.25, random_state=0)

# K-fold cross validation

In [110]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = LogisticRegression()
model.fit(X_train,y_train)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy (K-fold): %.3f (with SD: %.3f)' % (np.mean(scores), np.std(scores)))

C:\Users\nisha\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy (K-fold): 0.777 (with SD: 0.058)


In [111]:
y_pred = model.predict(X_test)

In [112]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [113]:
print('Confusion matrix: \n',confusion_matrix(y_test, y_pred))
print('Precision score: \n',precision_score(y_test, y_pred))

Confusion matrix: 
 [[36  9]
 [11 44]]
Precision score: 
 0.8301886792452831


In [160]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [164]:
cgm_value_stretch_meal_df_p2['class'] = 1

In [162]:
cgm_value_stretch_no_meal_df_p2['class'] = 0

In [180]:
df2 = cgm_value_stretch_meal_df_p2.iloc[:,-1]

In [179]:
df1 = cgm_value_stretch_meal_df_p2.iloc[:,:24]

In [183]:
frames = [df1, df2]

In [186]:
cgm_value_stretch_meal_df_p2 = pd.concat(frames, axis = 1)

In [193]:
cnn_data =  pd.concat([cgm_value_stretch_meal_df_p2, cgm_value_stretch_no_meal_df_p2], axis = 0)

In [196]:
X_train,X_test,y_train,y_test = train_test_split(cnn_data.iloc[:,:-1], cnn_data['class'], test_size=0.25, random_state=0)

In [197]:
from keras.utils import to_categorical
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Classification accuracy for CNN model

In [198]:
evaluate_model(X_train, y_train, X_test, y_test)

0.6294416245167631